In [1]:
!pip install numpy pandas sklearn openpyxl

In [4]:
import pandas as pd

# data location:
path = '../data/'
file_name = 'Online Retail.xlsx'
sheet_name = 'Online Retail'
# read excel file
df = pd.read_excel(path + file_name, sheet_name=sheet_name)

df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,`,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [5]:
# Drop unnecessary features
df.drop(columns=['Description', 'UnitPrice', 'Country'], inplace=True)

df

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID
0,536365,85123A,6,`,17850.0
1,536365,71053,6,2010-12-01 08:26:00,17850.0
2,536365,84406B,8,2010-12-01 08:26:00,17850.0
3,536365,84029G,6,2010-12-01 08:26:00,17850.0
4,536365,84029E,6,2010-12-01 08:26:00,17850.0
...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,12680.0
541905,581587,22899,6,2011-12-09 12:50:00,12680.0
541906,581587,23254,4,2011-12-09 12:50:00,12680.0
541907,581587,23255,4,2011-12-09 12:50:00,12680.0


In [6]:
# Split time and day
from datetime import datetime

# First clean the dates
def invalid_date(date):
    try:
        if isinstance(date, datetime):
            return date
        datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return date
    except Exception as e:
        print(e)
        return '??'

df['InvoiceDate'] = df.apply(lambda x: invalid_date(x['InvoiceDate']), axis=1)

df.loc[df['InvoiceDate']=='??', 'InvoiceDate'] = df.InvoiceDate.shift(-1)

def extract_date_data(date: str):
    if isinstance(date, datetime):
        date = date.strftime("%Y-%m-%d %H:%M:%S")
    date_time = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    day_start = datetime.strptime(date.split()[0] + " 00:00:00", "%Y-%m-%d %H:%M:%S")
    year_start = datetime.strptime(date.split()[0].split('-')[0] + "-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

    time_of_day = int((date_time - day_start).total_seconds()//60)
    day_of_year = (date_time - year_start).days

    return day_of_year, time_of_day



date_data = df['InvoiceDate'].apply(extract_date_data)

date_data_df = pd.DataFrame(date_data, columns=["Day", "Time"])

df[['Day', 'Time']] = pd.DataFrame(date_data.to_frame()['InvoiceDate'].to_list())

df

time data '`' does not match format '%Y-%m-%d %H:%M:%S'


,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Day,Time
0,536365,85123A,6,2010-12-01 08:26:00,17850.0,334,506
1,536365,71053,6,2010-12-01 08:26:00,17850.0,334,506
2,536365,84406B,8,2010-12-01 08:26:00,17850.0,334,506
3,536365,84029G,6,2010-12-01 08:26:00,17850.0,334,506
4,536365,84029E,6,2010-12-01 08:26:00,17850.0,334,506
...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,12680.0,342,770
541905,581587,22899,6,2011-12-09 12:50:00,12680.0,342,770
541906,581587,23254,4,2011-12-09 12:50:00,12680.0,342,770
541907,581587,23255,4,2011-12-09 12:50:00,12680.0,342,770


In [21]:
newdf = df

In [22]:
newdf.head(2)

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Day,Time
0,536365,85123A,6,2010-12-01 08:26:00,17850.0,334,506
1,536365,71053,6,2010-12-01 08:26:00,17850.0,334,506


In [25]:
# code for finding the quantity of repeating orders, try to subtract one from each row to find the repeating order

count_series = newdf.groupby(['CustomerID', 'StockCode']).size()
newdf_1 = count_series.to_frame(name = 'size').reset_index()

In [31]:
newdf_1.sort_values(['size'], ascending=[False]).head()

,CustomerID,StockCode,size
119844,14911.0,C2,85
251152,17841.0,79321,65
118697,14911.0,22423,56
250347,17841.0,21927,54
250362,17841.0,21975,54


In [10]:
# from itertools import combinations
# combos = newdf.groupby("CustomerID")["StockCode"].apply(lambda row: list(combinations(row, 2))).explode().value_counts()
# combos.reset_index()
# something = pd.DataFrame([[*x, y] for x, y in zip(combos.index, combos)], columns=["item_1", "item_2", "combo_qty"])
# something.head()

,item_1,item_2,combo_qty
0,85123A,85123A,370
1,85123A,82483,308
2,85123A,82494L,276
3,22423,22423,250
4,82483,82483,249


In [11]:
# combos
# something

,item_1,item_2,combo_qty
0,85123A,85123A,370
1,85123A,82483,308
2,85123A,82494L,276
3,22423,22423,250
4,82483,82483,249
...,...,...,...
1432590,20770,22332,1
1432591,20770,22497,1
1432592,20770,21910,1
1432593,20770,21911,1


In [13]:
# How many times the user reordered items
def reorders(customer_id):
    counts = df.loc[df['CustomerID']==customer_id].groupby('StockCode').count()
    # print(counts)
    counts['Quantity'] -= 1
    return counts['Quantity'].sum()

df['Reorders'] = df.apply(lambda x: reorders(x['CustomerID']), axis=1)


df

KeyboardInterrupt: 

In [ ]:
# How often is the product purchased
def orders_product(product_id):
    return len(df.loc[df['StockCode']==product_id].index)

df['TotalOrders'] = df.apply(lambda x: orders_product(x['StockCode']), axis=1)



In [ ]:
# Generate the target variable

In [ ]:
# Do the pca here


In [ ]:
# Model here, just use logistic regression